In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import tensorflow as tf
import os

2023-09-18 12:38:06.087931: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-18 12:38:06.745149: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [4]:
sequences_data = [
    [1, 2, 3],
    [4, 5],
    [6, 7, 8, 9],
]

labels_data = [0, 1, 0]

sequences_ragged = tf.ragged.constant(sequences_data, dtype=tf.int32)
labels_ragged = tf.ragged.constant(labels_data, dtype=tf.int32)

dataset = tf.data.Dataset.from_tensor_slices((sequences_ragged, labels_ragged))

2023-09-18 12:38:08.197326: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43192 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:81:00.0, compute capability: 8.6


In [5]:
batched_dataset = dataset.batch(batch_size=2)

In [6]:
def rand_data(x):
    return (tf.fill([x], x), tf.constant([1], dtype=tf.int32))

In [7]:
A = (tf.data.Dataset
     .range(1,5, output_type=tf.int32)
     .map(rand_data))
A = A.padded_batch(2)

In [11]:
input_layer = tf.keras.layers.Input(shape=(None,), dtype=tf.int32)
embedding_layer = tf.keras.layers.Embedding(input_dim=10, output_dim=32, mask_zero=False)(input_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(embedding_layer)

model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='binary_crossentropy', optimizer='adam')

epochs = 5
batch_size = 2

model.fit(batched_dataset, epochs=epochs)

Epoch 1/5
2/2 [==============================] - 1s 83ms/step - loss: 1.4673
Epoch 2/5
2/2 [==============================] - 0s 3ms/step - loss: 1.4655
Epoch 3/5
2/2 [==============================] - 0s 3ms/step - loss: 1.4637
Epoch 4/5
2/2 [==============================] - 0s 3ms/step - loss: 1.4619
Epoch 5/5
2/2 [==============================] - 0s 3ms/step - loss: 1.4601


In [10]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 32)          320       
                                                                 
 dense_1 (Dense)             (None, None, 1)           33        
                                                                 
Total params: 353 (1.38 KB)
Trainable params: 353 (1.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
